In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mne 
import os


In [9]:
def extract_eeg_data(folder_path, vhdr_file):
    """ Извлекает данные ЭЭГ, события и маркирует боль. """
    # Путь к файлу VHDR
    vhdr_path = os.path.join(folder_path, vhdr_file)

    # Загрузка данных с использованием MNE
    raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
    data = raw.get_data()  # Извлечение данных сигнала
    events, event_id = mne.events_from_annotations(raw)

    # Создание массива для временной шкалы, боли и напряжения
    time_array = [[i, 0, data[0][i]] for i in range(len(data[0]))]

    # Идентификаторы событий боли
    start_pain_id = 10001
    end_pain_id = 10003

    # Флаг для отслеживания боли
    flag = 0

    # Проход по массиву событий
    for event in events:
        if event[2] == start_pain_id:  # Начало боли
            flag = 1
        elif event[2] == end_pain_id:  # Конец боли
            flag = 0

        # Маркировка боли в массиве time_array
        if flag == 1:
            start_idx = event[0]  # Номер точки старта боли
            for i in range(start_idx, len(time_array)):
                time_array[i][1] = 1  # Маркировка боли
                if time_array[i][0] == event[0] and event[2] == end_pain_id:
                    break  # Прекращаем маркировку при достижении конца боли

    # Преобразование массива в DataFrame
    data_in_np = np.array(time_array)
    df2 = pd.DataFrame(data_in_np, columns=['Time', 'Pain', 'Voltage'])

    return df2

def save_eeg_images(df, step, resolution, patient_id, save_dir='dataset'):
    """ Сохраняет изображения ЭЭГ сигналов с болью и без боли в соответствующие папки. """
    pain_dir = os.path.join(save_dir, 'pain')
    no_pain_dir = os.path.join(save_dir, 'no_pain')

    # Создание папок, если их нет
    os.makedirs(pain_dir, exist_ok=True)
    os.makedirs(no_pain_dir, exist_ok=True)

    pain_segments = []
    no_pain_segments = []

    # Разделение данных на сегменты боли и без боли
    for i in range(len(df) // step):
        pain_marker = 0
        for k in range(i * step, (i + 1) * step):
            if df.iloc[k]['Pain'] == 1:
                pain_marker = 1
                break  # Если хотя бы одна точка содержит боль, прерываем цикл

        # Сохраняем индексы сегментов
        if pain_marker == 1:
            pain_segments.append(i)
        else:
            no_pain_segments.append(i)

    # Балансировка классов
    balanced_pain_segments = np.random.choice(pain_segments, len(no_pain_segments), replace=False)

    # Функция для сохранения изображений
    def save_plot(start_idx, folder, label):
        plt.plot(df['Voltage'][start_idx * step:(start_idx + 1) * step])
        plt.axis('off')
        plt.gcf().set_size_inches(resolution / 100, resolution / 100)
        file_name = f'{patient_id}_signal_{start_idx}_pain_{label}.png'
        file_path = os.path.join(folder, file_name)
        plt.savefig(file_path, dpi=100, format='png')
        plt.clf()  # Очищает текущую фигуру для следующего графика

    # Сохранение изображений с болью
    for idx in balanced_pain_segments:
        save_plot(idx, pain_dir, 1)

    # Сохранение изображений без боли
    for idx in no_pain_segments:
        save_plot(idx, no_pain_dir, 0)

    print(f"Изображения пациента {patient_id} успешно сохранены.")

def process_all_patients(folder_path, step=50, resolution=128):
    """ Обрабатывает данные для всех пациентов в папке и сохраняет изображения сигналов. """
    # Список всех VHDR файлов
    files = [f for f in os.listdir(folder_path) if f.endswith(".vhdr")]

    # Для каждого пациента (каждого набора файлов)
    for vhdr_file in files:
        # Извлечение идентификатора пациента из имени файла
        patient_id = vhdr_file.split('_')[-1].replace('.vhdr', '')
        print(f"Обработка данных для пациента {patient_id}...")

        # Извлечение данных ЭЭГ
        df = extract_eeg_data(folder_path, vhdr_file)

        # Сохранение изображений
        save_eeg_images(df, step, resolution, patient_id)



In [10]:
# Пример использования:
# Путь к папке с файлами всех пациентов
folder_path = 'experiment_3'
process_all_patients(folder_path)


Обработка данных для пациента vp01...
Extracting parameters from experiment_3\Exp_Mediation_Paradigm3_Motor_vp01.vhdr...
Setting channel info structure...
Reading 0 ... 715759  =      0.000 ...   715.759 secs...


C:\Users\George\AppData\Local\Temp\ipykernel_13112\3896280184.py:7: RuntimeWarning: Channels contain different highpass filters. Lowest (weakest) filter setting (0.00 Hz) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
C:\Users\George\AppData\Local\Temp\ipykernel_13112\3896280184.py:7: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (500.00 Hz, Nyquist limit) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)


Used Annotations descriptions: ['Laser/L  1', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  5', 'Stimulus/S  6', 'Stimulus/S  8', 'Stimulus/S  9', 'Toggle/T  2', 'Toggle/T  3']
Изображения пациента vp01 успешно сохранены.
Обработка данных для пациента vp02...
Extracting parameters from experiment_3\Exp_Mediation_Paradigm3_Motor_vp02.vhdr...
Setting channel info structure...
Reading 0 ... 660679  =      0.000 ...   660.679 secs...


C:\Users\George\AppData\Local\Temp\ipykernel_13112\3896280184.py:7: RuntimeWarning: Channels contain different highpass filters. Lowest (weakest) filter setting (0.00 Hz) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
C:\Users\George\AppData\Local\Temp\ipykernel_13112\3896280184.py:7: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (500.00 Hz, Nyquist limit) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)


Used Annotations descriptions: ['Laser/L  1', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  5', 'Stimulus/S  6', 'Stimulus/S  8', 'Stimulus/S  9', 'Toggle/T  2', 'Toggle/T  3']
Изображения пациента vp02 успешно сохранены.
Обработка данных для пациента vp03...
Extracting parameters from experiment_3\Exp_Mediation_Paradigm3_Motor_vp03.vhdr...
Setting channel info structure...
Reading 0 ... 639439  =      0.000 ...   639.439 secs...


C:\Users\George\AppData\Local\Temp\ipykernel_13112\3896280184.py:7: RuntimeWarning: Channels contain different highpass filters. Lowest (weakest) filter setting (0.00 Hz) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
C:\Users\George\AppData\Local\Temp\ipykernel_13112\3896280184.py:7: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (500.00 Hz, Nyquist limit) will be stored.
  raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)


Used Annotations descriptions: ['Laser/L  1', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  5', 'Stimulus/S  6', 'Stimulus/S  8', 'Stimulus/S  9', 'Toggle/T  2', 'Toggle/T  3']
Изображения пациента vp03 успешно сохранены.


<Figure size 128x128 with 0 Axes>